# Axisym PTST with unit ligament area

In [1]:
%matplotlib widget
import time
from bmcs_expsim.utils.mlab_decorators import decorate_figure
from mayavi import mlab
import numpy as np
np.seterr(divide='ignore', invalid='ignore') 
import warnings
import matplotlib.pylab as plt
import numpy as np

In [2]:
from ibvpy.api import TStepBC , TFCyclicNonsymmetricConstant, TFBilinear
from ibvpy.bcond import BCSlice, BCDof
from ibvpy.xmodel.xdomain_fe_grid_axisym import XDomainFEGridAxiSym
from ibvpy.xmodel.xdomain_interface import XDomainFEInterface

In [3]:
from ibvpy.fets import FETS2D4Q
from ibvpy.fets.fets1D5 import FETS1D52ULRH

In [4]:
from ibvpy.tmodel.viz3d_scalar_field import \
    Vis3DStateField, Viz3DScalarField
from ibvpy.tmodel.viz3d_tensor_field import \
    Vis3DTensorField, Viz3DTensorField

# Material models

In [5]:
from bmcs_matmod.slide.vslide_34_TN_axisym import Slide34
from ibvpy.tmodel.mats3D.mats3D_elastic.vmats3D_elastic import \
    MATS3DElastic

# FE axisymmetric discretization - Applying lateral compression/tension

## Area of the section = 1, load = 1

Expected $\sigma_{x} =  \frac{Load}{Area} =  \frac{1}{1} = 1$ 
Expected $u_x = \frac{\sigma_{x}}{E} = \frac{1}{1} = 1$ 

In [6]:
n_y_e = 8
n_inner_x_e = 4
n_outer_x_e = 4
L_x = 1.0
P = 1.0
R_in = P / (2 * np.pi) 
R_out = 2 * R_in
xd_lower = XDomainFEGridAxiSym(coord_min=(0, 0),
                          coord_max=(L_x, R_in),
                          shape=(n_y_e, n_inner_x_e),
                          integ_factor=2 * np.pi,
                          fets=FETS2D4Q())
xd_upper = XDomainFEGridAxiSym(coord_min=(0, R_in),
                          coord_max=(L_x, R_out),
                          shape=(n_y_e, n_outer_x_e),
                          integ_factor=2 * np.pi,
                          fets=FETS2D4Q())
m1 = MATS3DElastic(E=10000, nu=0.3)
m2 = MATS3DElastic(E=10000, nu=0.3)

xd12 = XDomainFEInterface(
    integ_factor = 2 * np.pi * R_in,
    I=xd_upper.mesh.I[:, -1],
    J=xd_lower.mesh.I[:, 0],
    fets=FETS1D52ULRH()
)

In [7]:
material_params =  dict(
     E_T=10000, gamma_T=500, K_T=0, S_T=0.5, c_T=6, bartau=3000, 
     E_N=10000, S_N=0.05, c_N = 3, m = 0., f_t=2000, f_c=100000, f_c0 = 40000, eta=0.)
bond_m = Slide34(**material_params)

In [8]:
axisym = BCSlice(slice=xd_lower.mesh[:, 0, :, 0],
                 var='u', dims=[1], value=0)
fixed_upper_support = BCSlice(slice=xd_upper.mesh[0, :, 0, :],
                 var='u', dims=[0], value=0)
radial = BCSlice(slice=xd_upper.mesh[:, -1, :, -1],
                 var='u', dims=[1], value=0.001)

F = -1 / len(radial.dofs)

radial_compression = [BCDof(var='f', dof=dof, value = F) 
                                      for dof in radial.dofs ]

bc1 = [axisym, fixed_upper_support ] + radial_compression

In [9]:
m = TStepBC(
    domains=[(xd_lower, m1),
             (xd_upper, m2),
             (xd12, bond_m),
             ],
    bc=bc1,  # + bc2,
)

m.hist.vis_record = {
#    'strain': Vis3DTensorField(var='eps_ab'),
    'stress': Vis3DTensorField(var='sig_ab'),
    #        'kinematic hardening': Vis3DStateField(var='z_a')
}

s = m.sim
s.tloop.verbose = True
s.tloop.k_max = 1000
s.tline.step = 0.1
s.tstep.fe_domain.serialized_subdomains

In [10]:
#bond_m.interact()

In [11]:
xd12.hidden = True
s.reset()
s.run()

t:	 0.00(0), 
	 0.10(1), 
	 0.20(1), 
	 0.30(1), 
	 0.40(1), 
	 0.50(1), 
	 0.60(1), 
	 0.70(1), 
	 0.80(1), 
	 0.90(1), 
	 1.00(1), 



In [12]:
states_t = [states_t[2] for states_t in m.hist.state_vars]
var_names = states_t[0].keys()
EpsSig_t = {
    var_name: np.array([ state_dict[var_name] for state_dict in states_t ]) 
    for var_name in var_names 
}
u_pi_N = EpsSig_t['w_pi']
u_pi_Tx = EpsSig_t['s_pi_x']
sig_pi_N = EpsSig_t['sig_pi']
sig_pi_Tx = EpsSig_t['tau_pi_x']
sig_pi_Ty = EpsSig_t['tau_pi_y']
omega_Nx = EpsSig_t['omega_N']
omega_Tx = EpsSig_t['omega_T']
x_m = xd12.x_Eia[:,:,0].flatten()

In [20]:
time = m.hist.t
_, (ax1, ax2) = plt.subplots(1,2)
ax1.plot(time, sig_pi_N[:,0,1].flatten(), label='sigma')
ax1.plot(time, sig_pi_N[:,1,1].flatten(), label='sigma')
ax1.set_xlabel('time')
ax1.set_ylabel('normal stress')
ax1.legend()

ax2.plot(time, sig_pi_Tx[:,0,0].flatten(), color='red', label='tau')
ax2.set_xlabel('time')
ax2.set_ylabel('tangential stress')
ax2.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [17]:
sig_pi_N[:,:,:].shape

(11, 4, 2)

In [14]:
if True:
    mlab.options.backend = 'envisage'
    mlab.options.offscreen = False # 'envisage'
    f_strain = mlab.figure()
    scene = mlab.get_engine().scenes[-1]
    scene.name = 'stress'
    strain_viz = Viz3DTensorField(vis3d=m.hist['stress'])
    strain_viz.setup()
    strain_viz.warp_vector.filter.scale_factor = 0.1
    strain_viz.plot(s.tstep.t_n)
    mlab.show()

# FE 2D discretization - Applying lateral compression/tension

## Area of the section = 1, load = 1

Expected $\sigma_{x} =  \frac{Load}{Area} =  \frac{1}{1} = 1$ 
Expected $u_x = \frac{\sigma_{x}}{E} = \frac{1}{1} = 1$ 

In [19]:
n_y_e = 8
n_inner_x_e = 4
n_outer_x_e = 4
L_x = 1.0
P = 1.0
R_in = P / (2 * np.pi) 
R_out = 2 * R_in
xd_lower = XDomainFEGridAxiSym(coord_min=(0, 0),
                          coord_max=(L_x, R_in),
                          shape=(n_y_e, n_inner_x_e),
                          fets=FETS2D4Q())
xd_upper = XDomainFEGridAxiSym(coord_min=(0, R_in),
                          coord_max=(L_x, R_out),
                          shape=(n_y_e, n_outer_x_e),
                          fets=FETS2D4Q())
m1 = MATS3DElastic(E=10000, nu=0.3)
m2 = MATS3DElastic(E=10000, nu=0.3)

xd12 = XDomainFEInterface(
    integ_factor = 2 * np.pi * R_in,
    I=xd_upper.mesh.I[:, -1],
    J=xd_lower.mesh.I[:, 0],
    fets=FETS1D52ULRH()
)

In [20]:
material_params =  dict(
     E_T=10000, gamma_T=500, K_T=0, S_T=0.5, c_T=6, bartau=3000, 
     E_N=10000, S_N=0.05, c_N = 3, m = 0., f_t=2000, f_c=100000, f_c0 = 40000, eta=0.)
bond_m = Slide34(**material_params)

In [28]:
lower_fixed = BCSlice(slice=xd_lower.mesh[:, :, :, :],
         var='u', dims=[0,1], value=0)
    
upper_compression_slice = BCSlice(slice=xd_upper.mesh[:, :, :, :],
         var='u', dims=[1], value=0)
compression_dofs = upper_compression_slice.dofs

compression = -100 / len(compression_dofs)


upper_compression_force_first = [BCDof(var='f', dof=dof, value = compression) 
             for dof in compression_dofs ]

bc1 =  upper_compression_force_first + [lower_fixed]

In [29]:
m = TStepBC(
    domains=[(xd_lower, m1),
             (xd_upper, m2),
             (xd12, bond_m),
             ],
    bc=bc1,  # + bc2,
)

m.hist.vis_record = {
#    'strain': Vis3DTensorField(var='eps_ab'),
    'stress': Vis3DTensorField(var='sig_ab'),
    #        'kinematic hardening': Vis3DStateField(var='z_a')
}

s = m.sim
s.tloop.verbose = True
s.tloop.k_max = 1000
s.tline.step = 0.1
s.tstep.fe_domain.serialized_subdomains

In [30]:
#bond_m.interact()

In [31]:
xd12.hidden = True
s.reset()
s.run()

t:	 0.00(0), 
	 0.10(1), 
	 0.20(1), 
	 0.30(1), 
	 0.40(1), 
	 0.50(1), 
	 0.60(1), 
	 0.70(1), 
	 0.80(1), 
	 0.90(1), 
	 1.00(1), 



In [32]:
states_t = [states_t[2] for states_t in m.hist.state_vars]
var_names = states_t[0].keys()
EpsSig_t = {
    var_name: np.array([ state_dict[var_name] for state_dict in states_t ]) 
    for var_name in var_names 
}
u_pi_N = EpsSig_t['w_pi']
u_pi_Tx = EpsSig_t['s_pi_x']
sig_pi_N = EpsSig_t['sig_pi']
sig_pi_Tx = EpsSig_t['tau_pi_x']
sig_pi_Ty = EpsSig_t['tau_pi_y']
omega_Nx = EpsSig_t['omega_N']
omega_Tx = EpsSig_t['omega_T']
x_m = xd12.x_Eia[:,:,0].flatten()

In [33]:
time = m.hist.t
_, (ax1, ax2) = plt.subplots(1,2)
ax1.plot(time, sig_pi_N[:,0,0].flatten(), label='sigma')
ax1.plot(time, sig_pi_N[:,0,1].flatten(), label='sigma')
ax1.set_xlabel('time')
ax1.set_ylabel('normal stress')
ax1.legend()

ax2.plot(time, sig_pi_Tx[:,0,0].flatten(), color='red', label='tau')
ax2.set_xlabel('time')
ax2.set_ylabel('tangential stress')
ax2.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [34]:
if True:
    mlab.options.backend = 'envisage'
    mlab.options.offscreen = False # 'envisage'
    f_strain = mlab.figure()
    scene = mlab.get_engine().scenes[-1]
    scene.name = 'stress'
    strain_viz = Viz3DTensorField(vis3d=m.hist['stress'])
    strain_viz.setup()
    strain_viz.warp_vector.filter.scale_factor = 0.1
    strain_viz.plot(s.tstep.t_n)
    mlab.show()